# NLP - Project 2
## Rinehart Analysis with Word Vectors
**Team**: *Jean Merlet, Konstantinos Georgiou, Matt Lane*

## Where to put the code
- Place the preprocessing functions/classes in [nlp_libs/books/preprocessing.py](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/nlp_libs/books/preprocessing.py)
- The custom word embeddings functions/classes (task 1) in [nlp_libs/books/word_embeddings.py](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/nlp_libs/books/word_embeddings.py) (separate class)
- The pretrained word embeddings functions/classes (task 2) in [nlp_libs/books/word_embeddings.py](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/nlp_libs/books/word_embeddings.py) (separate class)
- The functions/classes (if any) that compare the results (tasks 3, 4, 5) in [nlp_libs/books/compare_statistics.py](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/nlp_libs/books/compare_statistics.py)
- Any plotting related functions in [nlp_libs/books/plotter.py](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/nlp_libs/books/plotter.py)

**The code is reloaded automatically. Any class object needs to reinitialized though.** 

## Config file
The yml/config file is located at: [confs/proj_2.yml](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/confs/proj_2.yml)<br>
To load it run:
```python
config_path='confs/proj_2.yml'
conf = Configuration(config_src=config_path)
# Get the books dictionary
books = conf.get_config('data_loader')['config']['books'] # type = Dict
print(books.keys())
print(books['The_Bat'])
```
To reload the config just run the 2nd and 3rd command.

## Libraries Overview:
All the libraries are located under *"\<project root>/nlp_libs"*
- nlp_libs/**books**: This project's code (imported later)
- nlp_libs/**configuration**: Class that creates config objects from yml files
- nlp_libs/**fancy_logger**: Logger that can be used instead of prints for text formatting (color, bold, underline etc)

## Project 1 Code
If you need to import anything from Project 1 just run:
```python
import proj1_nlp_libs.books.processed_book as proc
import proj1_nlp_libs.books.book_extractor as extr
import proj1_nlp_libs.books.plotter as pl
```

## For more info check out:
- the **[Project Board](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/projects/1)**
- the **[README](https://github.com/NLPaladins/https://github.com/NLPaladins/rinehartAnalysis_wordVectors/blob/main/README.md)**
- and the **[Current Issues](https://github.com/NLPaladins/rinehartAnalysis_wordVectors/issues)**

# ------------------------------------------------------------------

## On Google Collab?
- **If yes, run the two cells and press the two buttons below:**
- Otherwise go to "***Import the base Libraries***"

In [1]:
# Import Jupyter Widgets
import os
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
# Clone the repository if you're in Google Collab
def clone_project(is_collab: bool = False):
    print("Cloning Project..")
    !git clone https://github.com/NLPaladins/rinehartAnalysis_wordVectors.git
    print("Project cloned.")
       
print("Clone project?")
print("(If you do this you will ovewrite local changes on other files e.g. configs)")
print("Not needed if you're not on Google Collab")
btn = widgets.Button(description="Yes, clone")
btn.on_click(clone_project)
display(btn)

Clone project?
(If you do this you will ovewrite local changes on other files e.g. configs)
Not needed if you're not on Google Collab


Button(description='Yes, clone', style=ButtonStyle())

In [2]:
# Clone the repository if you're in Google Collab
def change_dir(is_collab: bool = False):
    try:
        print("Changing dir..")
        os.chdir('/content/rinehartAnalysis')
        print('done')
        print("Current dir:")
        print(os.getcwd())
        print("Dir Contents:")
        print(os.listdir())
        print("\nInstalling Requirements")
        !pip install -r requirements.txt
    except Exception:
        print("Error: Project not cloned")
       
print("Are you on Google Collab?")
btn = widgets.Button(description="Yes")
btn.on_click(change_dir)
display(btn)

Are you on Google Collab?


Button(description='Yes', style=ButtonStyle())

### To commit and push Google Collab notebook to Github
Click **File > Save a copy on Gihtub**

# ------------------------------------------------------------------

# Initializations

## Import the base Libraries

In [35]:
# Imports
%load_ext autoreload
%autoreload 2
from importlib import reload as reload_lib
from typing import *
import os
import re
from pprint import pprint
# Numpy
import numpy as np

# Import preprocessing lib
from nlp_libs.books import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the YML file

In [11]:
from nlp_libs import Configuration

In [12]:
# The path of configuration and log save path
config_path = "confs/proj_2.yml"
# !cat "$config_path"
# Load the configuration
conf = Configuration(config_src=config_path)
# Get the books dict
books = conf.get_config('data_loader')['config']['books']
# print(books.keys())
# pprint(books)  # Pretty print the books dict

2021-10-31 17:52:18 Config       INFO     Configuration file loaded successfully from path: /home/jeanmerlet/projects/nlp/rinehartAnalysis_wordVectors/confs/proj_2.yml
2021-10-31 17:52:18 Config       INFO     Configuration Tag: proj2


## Setup Logger and Example

In [6]:
log_path = "logs/proj_2.log"
# Load and setup logger
logger = ColorizedLogger(logger_name='Notebook', color='cyan')
ColorizedLogger.setup_logger(log_path=log_path, debug=False, clear_log=True)
# Examples
logger.info("Logger Examples:")
logger.nl(num_lines=1) # New lines
logger.warn("Logger Warning underlined", attrs=['underline']) 
# Atrs:  bold, dark, underline, blink, reverse, concealed
logger.error("Logger Error in red&yellow", color="yellow", on_color="on_red")
# Colors: on_grey, on_red, on_green, on_yellow, on_blue, on_magenta, on_cyan, on_white

2021-10-31 17:51:16 FancyLogger  INFO     Logger is set. Log file path: /home/jeanmerlet/projects/nlp/rinehartAnalysis_wordVectors/logs/proj_2.log
2021-10-31 17:51:16 Notebook     INFO     Logger Examples:

2021-10-31 17:51:16 Notebook     WARNING  Logger Warning underlined
2021-10-31 17:51:16 Notebook     ERROR    Logger Error in red&yellow


# ------------------------------------------------------------------

# Start of Project Code

## Preprocessing

# The Circular Staircase

In [36]:
book_meta = books['The_Circular_Staircase']
book = ProcessedBook(book_meta)

In [42]:
protagonist_subs = list(book_meta['protagonists'][0].values())[0]
substitution = (protagonist_subs, 'protagonist')
sentences = book.lemmatize_by_sentence(word_subs=substitution)

In [15]:
df = books.word_embeddings.calculate_differing_distances(output, [['jamieson', 'watson'], 
                                                            ['revolver', 'watson'], 
                                                            ['murder', 'watson'], 
                                                            ['jamieson', 'murder'], 
                                                            ['jamieson', 'revolver'],
                                                            ['watson', 'revolver'],
                                                            ['murder', 'revolver'],
                                                            ['murder', 'bag'],
                                                            ['murder', 'rachel'],                                                                  
                                                            ['jamieson', 'detective']])

AttributeError: 'dict' object has no attribute 'word_embeddings'

In [76]:
df.sort_values(['cosineSim', 'dotSim'])

,word1,word2,vectorSize,windowSize,cosineSim,dotSim
97,murder,bag,200,5,0.995936,0.625594
107,murder,bag,200,3,0.996555,0.718513
27,murder,bag,50,3,0.987875,0.775404
147,murder,bag,300,3,0.998056,0.812726
67,murder,bag,100,3,0.995341,0.831142
...,...,...,...,...,...,...
139,jamieson,detective,300,5,0.999730,6.556924
39,jamieson,detective,50,10,0.998634,7.786859
159,jamieson,detective,300,10,0.999756,7.862607
79,jamieson,detective,100,10,0.999016,8.192954


# The Man in the Lower Ten

In [ ]:
book = ProcessedBook(books['The_Man_in_Lower_Ten'])
lemmas = book.lemmas

In [ ]:
' '.join(lemmas[:100])

In [ ]:
book.clean_lines

# The After House

In [ ]:
book = ProcessedBook(books['The_After_House'])
lemmas = book.lemmas

In [ ]:
' '.join(lemmas[:100])

# The Window at the Wide Cat

In [ ]:
book = ProcessedBook(books['The_Window_at_the_White_Cat'])
lemmas = book.lemmas

In [ ]:
' '.join(lemmas[:100])

# The Bat

In [ ]:
book = ProcessedBook(books['The_Bat'])
lemmas = book.lemmas

In [ ]:
' '.join(lemmas[:100])

## Custom Word Embeddings

In [ ]:
# Import word_embeddings lib
import nlp_libs.books.word_embeddings as we

In [ ]:
# custom_embeddings = we.WordEmbeddingsCustom()

## Pretrained Word Embeddings

In [ ]:
# pretrained_embeddings = we.WordEmbeddingsPretrained()

## Compare Vector distances and report similarities using Custom Embeddings

In [ ]:
# Import compare_statistics lib
import nlp_libs.books.compare_statistics as cs

In [ ]:
# cs.my_custom_embeddings_compare_function()

## Compare Vector distances and report similarities using Pretrained Embeddings

In [ ]:
# cs.my_pretrained_embeddings_compare_function()

## Extra Analysis? Plots?

In [ ]:
# Too much work